In [5]:
import pandas as pd


def clean_book_data(book_file_path):
    # Load the book data
    book_data = pd.read_csv(book_file_path)

    # Convert 'Book checkout' and 'Book Returned' columns to datetime, handle invalid formats
    book_data['Book checkout'] = pd.to_datetime(book_data['Book checkout'].str.replace('"', ''), errors='coerce')
    book_data['Book Returned'] = pd.to_datetime(book_data['Book Returned'].str.replace('"', ''), errors='coerce')

    # Handle missing date values by dropping rows with missing dates
    book_data_cleaned = book_data.dropna(subset=['Book checkout', 'Book Returned'])

    # Normalize book titles (strip whitespace, fix capitalization)
    book_data_cleaned['Books'] = book_data_cleaned['Books'].str.strip().str.title()

    # Calculate loan duration in days
    book_data_cleaned['Loan Duration (Days)'] = (book_data_cleaned['Book Returned'] - book_data_cleaned['Book checkout']).dt.days

    # Handle negative loan durations (set to NaN for further review)
    book_data_cleaned['Loan Duration (Days)'] = book_data_cleaned['Loan Duration (Days)'].apply(lambda x: x if x >= 0 else None)

    return book_data_cleaned

def clean_customer_data(customer_file_path):
    # Load the customer data
    customer_data = pd.read_csv(customer_file_path)

    # Drop rows with missing customer information
    customer_data_cleaned = customer_data.dropna(subset=['Customer ID', 'Customer Name'])

    return customer_data_cleaned

import os

def main():
    # Define file paths (replace with actual file paths as needed)
    book_file_path = r'C:\Users\Admin\Desktop\DE5_InputFiles\03_Library Systembook.csv'  # Update with your file path
    customer_file_path = r'C:\Users\Admin\Desktop\DE5_InputFiles\03_Library SystemCustomers.csv'  # Update with your file path

    # Check the current working directory
    print(f"Current working directory: {os.getcwd()}")

    # Clean the data
    cleaned_book_data = clean_book_data(book_file_path)
    cleaned_customer_data = clean_customer_data(customer_file_path)

    # Save cleaned data to new CSV files (with full file paths if necessary)
    cleaned_book_data.to_csv('cleaned_books_data.csv', index=False)
    cleaned_customer_data.to_csv('cleaned_customers_data.csv', index=False)

    print("Data cleaning complete. Cleaned data saved to 'cleaned_books_data.csv' and 'cleaned_customers_data.csv'.")

if __name__ == "__main__":
    main()


Current working directory: c:\Users\Admin\Desktop\DE5_Module
Data cleaning complete. Cleaned data saved to 'cleaned_books_data.csv' and 'cleaned_customers_data.csv'.


C:\Users\Admin\AppData\Local\Temp\ipykernel_13924\1341906620.py:9: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  book_data['Book checkout'] = pd.to_datetime(book_data['Book checkout'].str.replace('"', ''), errors='coerce')
C:\Users\Admin\AppData\Local\Temp\ipykernel_13924\1341906620.py:10: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  book_data['Book Returned'] = pd.to_datetime(book_data['Book Returned'].str.replace('"', ''), errors='coerce')
C:\Users\Admin\AppData\Local\Temp\ipykernel_13924\1341906620.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing